In [1]:
# x86_desktop 1.5GHz -> 3.0GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['br_inst_retired.all_branches_pebs:u'],
                row['br_misp_retired.all_branches_pebs:u'],
                row['L1-dcache-load-misses:u'],
                row['L1-dcache-loads:u'],
                row['L1-dcache-stores:u'],
                row['L1-icache-load-misses:u'],
                row['LLC-load-misses:u'],
                row['LLC-loads:u'],
                row['LLC-store-misses:u'],
                row['LLC-stores:u'],
                row['fp_arith_inst_retired.scalar_single:u'],
                row['dTLB-load-misses:u'],
                row['dTLB-loads:u'],
                row['dTLB-store-misses:u'],
                row['dTLB-stores:u'],
                row['iTLB-load-misses:u'],
                row['iTLB-loads:u'],
                row['mem_load_retired.l1_hit:u'],
                row['mem_load_retired.l1_miss:u'],
                row['mem_load_retired.l2_hit:u'],
                row['mem_load_retired.l2_miss:u'],
                row['mem_load_retired.l3_hit:u'],
                row['mem_load_retired.l3_miss:u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(one_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(three_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'br_inst_retired.all_branches_pebs:u',
            'br_misp_retired.all_branches_pebs:u',
            'L1-dcache-load-misses:u',
            'L1-dcache-loads:u',
            'L1-dcache-stores:u',
            'L1-icache-load-misses:u',
            'LLC-load-misses:u',
            'LLC-loads:u',
            'LLC-store-misses:u',
            'LLC-stores:u',
            'fp_arith_inst_retired.scalar_single:u',
            'dTLB-load-misses:u',
            'dTLB-loads:u',
            'dTLB-store-misses:u',
            'dTLB-stores:u',
            'iTLB-load-misses:u',
            'iTLB-loads:u',
            'mem_load_retired.l1_hit:u',
            'mem_load_retired.l1_miss:u',
            'mem_load_retired.l2_hit:u',
            'mem_load_retired.l2_miss:u',
            'mem_load_retired.l3_hit:u',
            'mem_load_retired.l3_miss:u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.7288514296052259
Testing R²: 0.7125425241828213
Training Mape: 0.14513365169168363
Testing Mape: 0.14843242405992274
chunk_size: 2000000000
Training R²: 0.7700310389596328
Testing R²: 0.7495478610479038
Training Mape: 0.12902940919604197
Testing Mape: 0.13734850635754933
chunk_size: 3000000000
Training R²: 0.7980342684387793
Testing R²: 0.7499905736683302
Training Mape: 0.12065313859993575
Testing Mape: 0.12617922371390602
chunk_size: 4000000000
Training R²: 0.8139635565625302
Testing R²: 0.7891770286219818
Training Mape: 0.11458383906661605
Testing Mape: 0.1251985558476812
chunk_size: 5000000000
Training R²: 0.8298103013539355
Testing R²: 0.7702492485081788
Training Mape: 0.10897151454009
Testing Mape: 0.12036218571758871
chunk_size: 10000000000
Training R²: 0.8617436296705387
Testing R²: 0.7671928421253837
Training Mape: 0.09609822768386724
Testing Mape: 0.11989341549807665
chunk_size: 20000000000
Training R²: 0.8972829802615504
Testing R²: 0.760

In [2]:
# x86_desktop 3.0GHz -> 1.5GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['br_inst_retired.all_branches_pebs:u'],
                row['br_misp_retired.all_branches_pebs:u'],
                row['L1-dcache-load-misses:u'],
                row['L1-dcache-loads:u'],
                row['L1-dcache-stores:u'],
                row['L1-icache-load-misses:u'],
                row['LLC-load-misses:u'],
                row['LLC-loads:u'],
                row['LLC-store-misses:u'],
                row['LLC-stores:u'],
                row['fp_arith_inst_retired.scalar_single:u'],
                row['dTLB-load-misses:u'],
                row['dTLB-loads:u'],
                row['dTLB-store-misses:u'],
                row['dTLB-stores:u'],
                row['iTLB-load-misses:u'],
                row['iTLB-loads:u'],
                row['mem_load_retired.l1_hit:u'],
                row['mem_load_retired.l1_miss:u'],
                row['mem_load_retired.l2_hit:u'],
                row['mem_load_retired.l2_miss:u'],
                row['mem_load_retired.l3_hit:u'],
                row['mem_load_retired.l3_miss:u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(three_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(one_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'br_inst_retired.all_branches_pebs:u',
            'br_misp_retired.all_branches_pebs:u',
            'L1-dcache-load-misses:u',
            'L1-dcache-loads:u',
            'L1-dcache-stores:u',
            'L1-icache-load-misses:u',
            'LLC-load-misses:u',
            'LLC-loads:u',
            'LLC-store-misses:u',
            'LLC-stores:u',
            'fp_arith_inst_retired.scalar_single:u',
            'dTLB-load-misses:u',
            'dTLB-loads:u',
            'dTLB-store-misses:u',
            'dTLB-stores:u',
            'iTLB-load-misses:u',
            'iTLB-loads:u',
            'mem_load_retired.l1_hit:u',
            'mem_load_retired.l1_miss:u',
            'mem_load_retired.l2_hit:u',
            'mem_load_retired.l2_miss:u',
            'mem_load_retired.l3_hit:u',
            'mem_load_retired.l3_miss:u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.6853700661935851
Testing R²: 0.6644641835622694
Training Mape: 0.15317778916182495
Testing Mape: 0.15729843547677647
chunk_size: 2000000000
Training R²: 0.7353729883794378
Testing R²: 0.7010200400535095
Training Mape: 0.13141688067630813
Testing Mape: 0.13922242441797936
chunk_size: 3000000000
Training R²: 0.7580716839843188
Testing R²: 0.7168819399110915
Training Mape: 0.12366369904259254
Testing Mape: 0.12891754904713373
chunk_size: 4000000000
Training R²: 0.7800921147514553
Testing R²: 0.7351137950002362
Training Mape: 0.11649819286434561
Testing Mape: 0.12951000596758308
chunk_size: 5000000000
Training R²: 0.7934857983307623
Testing R²: 0.7315787066874224
Training Mape: 0.11125158774324805
Testing Mape: 0.13142161140346806
chunk_size: 10000000000
Training R²: 0.8409646396216777
Testing R²: 0.7501634826929977
Training Mape: 0.09912764836881673
Testing Mape: 0.11497005037613166
chunk_size: 20000000000
Training R²: 0.8912573116814652
Testing R²: 0

In [3]:
# x86_ecore 1.5GHz -> 3.0GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_atom/instructions/u']
            cyc = row['cpu_atom/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_atom/instructions/u'],
            row['cpu_atom/cpu-cycles/u'],
            row['cpu_atom/br_inst_retired.all_branches/u'],
            row['cpu_atom/br_misp_retired.all_branches/u'],
            row['cpu_atom/cache-misses/u'],
            row['cpu_atom/L1-dcache-loads:u/'],
            row['cpu_atom/L1-dcache-stores:u/'],
            row['cpu_atom/L1-icache-load-misses:u/'],
            row['cpu_atom/LLC-load-misses:u/'],
            row['cpu_atom/LLC-loads:u/'],
            row['cpu_atom/LLC-store-misses:u/'],
            row['cpu_atom/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single:u/'],
            row['cpu_atom/dTLB-load-misses:u/'],
            row['cpu_atom/dTLB-loads:u/'],
            row['cpu_atom/dTLB-store-misses:u/'],
            row['cpu_atom/dTLB-stores:u/'],
            row['cpu_atom/iTLB-load-misses:u/'],
            row['cpu_atom/bus-cycles/u'],
            row['cpu_core/mem_load_retired.l1_hit:u/'],
            row['cpu_core/mem_load_retired.l1_miss:u/'],
            row['cpu_core/mem_load_retired.l2_hit:u/'],
            row['cpu_core/mem_load_retired.l2_miss:u/'],
            row['cpu_core/mem_load_retired.l3_hit:u/'],
            row['cpu_core/mem_load_retired.l3_miss:u/'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_atom/instructions/u'] / row['cpu_atom/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(one_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(three_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_atom/instructions/u',
            'cpu_atom/cpu-cycles/u',
            'cpu_atom/br_inst_retired.all_branches/u',
            'cpu_atom/br_misp_retired.all_branches/u',
            'cpu_atom/cache-misses/u',
            'cpu_atom/L1-dcache-loads:u/',
            'cpu_atom/L1-dcache-stores:u/',
            'cpu_atom/L1-icache-load-misses:u/',
            'cpu_atom/LLC-load-misses:u/',
            'cpu_atom/LLC-loads:u/',
            'cpu_atom/LLC-store-misses:u/',
            'cpu_atom/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single:u/',
            'cpu_atom/dTLB-load-misses:u/',
            'cpu_atom/dTLB-loads:u/',
            'cpu_atom/dTLB-store-misses:u/',
            'cpu_atom/dTLB-stores:u/',
            'cpu_atom/iTLB-load-misses:u/',
            'cpu_atom/bus-cycles/u',
            'cpu_core/mem_load_retired.l1_hit:u/',
            'cpu_core/mem_load_retired.l1_miss:u/',
            'cpu_core/mem_load_retired.l2_hit:u/',
            'cpu_core/mem_load_retired.l2_miss:u/',
            'cpu_core/mem_load_retired.l3_hit:u/',
            'cpu_core/mem_load_retired.l3_miss:u/',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.6369577990970214
Testing R²: 0.6303160202137622
Training Mape: 0.25982861071958796
Testing Mape: 0.26179183119236266
chunk_size: 2000000000
Training R²: 0.7146827901350649
Testing R²: 0.6875782411475269
Training Mape: 0.21040643613943447
Testing Mape: 0.22522178308176374
chunk_size: 3000000000
Training R²: 0.7292681867905628
Testing R²: 0.7087220034694488
Training Mape: 0.20865998086767804
Testing Mape: 0.21369865712167085
chunk_size: 4000000000
Training R²: 0.755926718348208
Testing R²: 0.7053535512376228
Training Mape: 0.19544865148728083
Testing Mape: 0.20822261778192064
chunk_size: 5000000000
Training R²: 0.7488264687434707
Testing R²: 0.7058361985384507
Training Mape: 0.198297377241462
Testing Mape: 0.22445612881834828
chunk_size: 10000000000
Training R²: 0.7945581840009794
Testing R²: 0.7187224455701922
Training Mape: 0.18251715114171135
Testing Mape: 0.19668774348235182
chunk_size: 20000000000
Training R²: 0.8236693085064172
Testing R²: 0.72

In [4]:
# x86_ecore 3.0GHz -> 1.5GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_atom/instructions/u']
            cyc = row['cpu_atom/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_atom/instructions/u'],
            row['cpu_atom/cpu-cycles/u'],
            row['cpu_atom/br_inst_retired.all_branches/u'],
            row['cpu_atom/br_misp_retired.all_branches/u'],
            row['cpu_atom/cache-misses/u'],
            row['cpu_atom/L1-dcache-loads:u/'],
            row['cpu_atom/L1-dcache-stores:u/'],
            row['cpu_atom/L1-icache-load-misses:u/'],
            row['cpu_atom/LLC-load-misses:u/'],
            row['cpu_atom/LLC-loads:u/'],
            row['cpu_atom/LLC-store-misses:u/'],
            row['cpu_atom/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single:u/'],
            row['cpu_atom/dTLB-load-misses:u/'],
            row['cpu_atom/dTLB-loads:u/'],
            row['cpu_atom/dTLB-store-misses:u/'],
            row['cpu_atom/dTLB-stores:u/'],
            row['cpu_atom/iTLB-load-misses:u/'],
            row['cpu_atom/bus-cycles/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_atom/instructions/u'] / row['cpu_atom/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(three_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(one_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_atom/instructions/u',
            'cpu_atom/cpu-cycles/u',
            'cpu_atom/br_inst_retired.all_branches/u',
            'cpu_atom/br_misp_retired.all_branches/u',
            'cpu_atom/cache-misses/u',
            'cpu_atom/L1-dcache-loads:u/',
            'cpu_atom/L1-dcache-stores:u/',
            'cpu_atom/L1-icache-load-misses:u/',
            'cpu_atom/LLC-load-misses:u/',
            'cpu_atom/LLC-loads:u/',
            'cpu_atom/LLC-store-misses:u/',
            'cpu_atom/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single:u/',
            'cpu_atom/dTLB-load-misses:u/',
            'cpu_atom/dTLB-loads:u/',
            'cpu_atom/dTLB-store-misses:u/',
            'cpu_atom/dTLB-stores:u/',
            'cpu_atom/iTLB-load-misses:u/',
            'cpu_atom/bus-cycles/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.6349860254023201
Testing R²: 0.6223392934570424
Training Mape: 0.24727609317675564
Testing Mape: 0.24864734066348126
chunk_size: 2000000000
Training R²: 0.7171875015756753
Testing R²: 0.6804884245837504
Training Mape: 0.19774288605409135
Testing Mape: 0.20763157727517662
chunk_size: 3000000000
Training R²: 0.7269927483292449
Testing R²: 0.6951348601590788
Training Mape: 0.19597630276103445
Testing Mape: 0.20317336982391312
chunk_size: 4000000000
Training R²: 0.7492691904924244
Testing R²: 0.7006939077539844
Training Mape: 0.1815193446932073
Testing Mape: 0.1945370489892773
chunk_size: 5000000000
Training R²: 0.7313146554413451
Testing R²: 0.7017597916749205
Training Mape: 0.1938013342074025
Testing Mape: 0.19773965724849518
chunk_size: 10000000000
Training R²: 0.7776908079690913
Testing R²: 0.6917810487677623
Training Mape: 0.17356428144008823
Testing Mape: 0.198397745159424
chunk_size: 20000000000
Training R²: 0.8142129672915978
Testing R²: 0.7022

In [5]:
# x86_ecore 1.5GHz -> 3.0GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_core/instructions/u']
            cyc = row['cpu_core/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_core/instructions/u'],
            row['cpu_core/cpu-cycles/u'],
            row['cpu_core/br_inst_retired.all_branches/u'],
            row['cpu_core/br_misp_retired.all_branches/u'],
            row['cpu_core/L1-dcache-load-misses:u/'],
            row['cpu_core/L1-dcache-loads:u/'],
            row['cpu_core/L1-dcache-stores:u/'],
            row['cpu_core/L1-icache-load-misses:u/'],
            row['cpu_core/LLC-load-misses:u/'],
            row['cpu_core/LLC-loads:u/'],
            row['cpu_core/LLC-store-misses:u/'],
            row['cpu_core/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single/u'],
            row['cpu_core/dTLB-load-misses:u/'],
            row['cpu_core/dTLB-loads:u/'],
            row['cpu_core/dTLB-store-misses:u/'],
            row['cpu_core/dTLB-stores:u/'],
            row['cpu_core/iTLB-load-misses:u/'],
            row['cpu_core/bus-cycles/u'],
            row['cpu_core/mem_load_retired.l1_hit/u'],
            row['cpu_core/mem_load_retired.l1_miss/u'],
            row['cpu_core/mem_load_retired.l2_hit/u'],
            row['cpu_core/mem_load_retired.l2_miss/u'],
            row['cpu_core/mem_load_retired.l3_hit/u'],
            row['cpu_core/mem_load_retired.l3_miss/u'],
            ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_core/instructions/u'] / row['cpu_core/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(one_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(three_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_core/instructions/u',
            'cpu_core/cpu-cycles/u',
            'cpu_core/br_inst_retired.all_branches/u',
            'cpu_core/br_misp_retired.all_branches/u',
            'cpu_core/L1-dcache-load-misses:u/',
            'cpu_core/L1-dcache-loads:u/',
            'cpu_core/L1-dcache-stores:u/',
            'cpu_core/L1-icache-load-misses:u/',
            'cpu_core/LLC-load-misses:u/',
            'cpu_core/LLC-loads:u/',
            'cpu_core/LLC-store-misses:u/',
            'cpu_core/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single/u',
            'cpu_core/dTLB-load-misses:u/',
            'cpu_core/dTLB-loads:u/',
            'cpu_core/dTLB-store-misses:u/',
            'cpu_core/dTLB-stores:u/',
            'cpu_core/iTLB-load-misses:u/',
            'cpu_core/bus-cycles/u',
            'cpu_core/mem_load_retired.l1_hit/u',
            'cpu_core/mem_load_retired.l1_miss/u',
            'cpu_core/mem_load_retired.l2_hit/u',
            'cpu_core/mem_load_retired.l2_miss/u',
            'cpu_core/mem_load_retired.l3_hit/u',
            'cpu_core/mem_load_retired.l3_miss/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.6339327441472986
Testing R²: 0.6269649059760245
Training Mape: 0.1796526073388701
Testing Mape: 0.18347735240057367
chunk_size: 2000000000
Training R²: 0.713434088290628
Testing R²: 0.6620666224357241
Training Mape: 0.15230510719359144
Testing Mape: 0.16282443270168548
chunk_size: 3000000000
Training R²: 0.7320382220327466
Testing R²: 0.6710296008309995
Training Mape: 0.14503224492913327
Testing Mape: 0.15926127301547405
chunk_size: 4000000000
Training R²: 0.7520331267610827
Testing R²: 0.7025312200100082
Training Mape: 0.13905071066116653
Testing Mape: 0.15210866517489707
chunk_size: 5000000000
Training R²: 0.7678666072567712
Testing R²: 0.6985897213945189
Training Mape: 0.13394847891416053
Testing Mape: 0.14686689511786574
chunk_size: 10000000000
Training R²: 0.7846918708144959
Testing R²: 0.7298712984599116
Training Mape: 0.1254457729715296
Testing Mape: 0.1425942450005778
chunk_size: 20000000000
Training R²: 0.8406197223684363
Testing R²: 0.722

In [6]:
# x86_ecore 3.0GHz -> 1.5GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_core/instructions/u']
            cyc = row['cpu_core/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_core/instructions/u'],
            row['cpu_core/cpu-cycles/u'],
            row['cpu_core/br_inst_retired.all_branches/u'],
            row['cpu_core/br_misp_retired.all_branches/u'],
            row['cpu_core/L1-dcache-load-misses:u/'],
            row['cpu_core/L1-dcache-loads:u/'],
            row['cpu_core/L1-dcache-stores:u/'],
            row['cpu_core/L1-icache-load-misses:u/'],
            row['cpu_core/LLC-load-misses:u/'],
            row['cpu_core/LLC-loads:u/'],
            row['cpu_core/LLC-store-misses:u/'],
            row['cpu_core/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single/u'],
            row['cpu_core/dTLB-load-misses:u/'],
            row['cpu_core/dTLB-loads:u/'],
            row['cpu_core/dTLB-store-misses:u/'],
            row['cpu_core/dTLB-stores:u/'],
            row['cpu_core/iTLB-load-misses:u/'],
            row['cpu_core/bus-cycles/u'],
            row['cpu_core/mem_load_retired.l1_hit/u'],
            row['cpu_core/mem_load_retired.l1_miss/u'],
            row['cpu_core/mem_load_retired.l2_hit/u'],
            row['cpu_core/mem_load_retired.l2_miss/u'],
            row['cpu_core/mem_load_retired.l3_hit/u'],
            row['cpu_core/mem_load_retired.l3_miss/u'],
            ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_core/instructions/u'] / row['cpu_core/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(three_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(one_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_core/instructions/u',
            'cpu_core/cpu-cycles/u',
            'cpu_core/br_inst_retired.all_branches/u',
            'cpu_core/br_misp_retired.all_branches/u',
            'cpu_core/L1-dcache-load-misses:u/',
            'cpu_core/L1-dcache-loads:u/',
            'cpu_core/L1-dcache-stores:u/',
            'cpu_core/L1-icache-load-misses:u/',
            'cpu_core/LLC-load-misses:u/',
            'cpu_core/LLC-loads:u/',
            'cpu_core/LLC-store-misses:u/',
            'cpu_core/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single/u',
            'cpu_core/dTLB-load-misses:u/',
            'cpu_core/dTLB-loads:u/',
            'cpu_core/dTLB-store-misses:u/',
            'cpu_core/dTLB-stores:u/',
            'cpu_core/iTLB-load-misses:u/',
            'cpu_core/bus-cycles/u',
            'cpu_core/mem_load_retired.l1_hit/u',
            'cpu_core/mem_load_retired.l1_miss/u',
            'cpu_core/mem_load_retired.l2_hit/u',
            'cpu_core/mem_load_retired.l2_miss/u',
            'cpu_core/mem_load_retired.l3_hit/u',
            'cpu_core/mem_load_retired.l3_miss/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.6495756320329983
Testing R²: 0.6342540089463131
Training Mape: 0.16720499712297335
Testing Mape: 0.1677956736571613
chunk_size: 2000000000
Training R²: 0.685630417400583
Testing R²: 0.6310668120140317
Training Mape: 0.14939154077694664
Testing Mape: 0.16303659003932228
chunk_size: 3000000000
Training R²: 0.6955687854539809
Testing R²: 0.6419246749918768
Training Mape: 0.15023509859070727
Testing Mape: 0.15663912151090667
chunk_size: 4000000000
Training R²: 0.7168189728094847
Testing R²: 0.6602558008386885
Training Mape: 0.14154004870060194
Testing Mape: 0.15757964194563234
chunk_size: 5000000000
Training R²: 0.7292307905064004
Testing R²: 0.6547455764744544
Training Mape: 0.13977092714415965
Testing Mape: 0.15683386125056484
chunk_size: 10000000000
Training R²: 0.7968753995903464
Testing R²: 0.6460689603186045
Training Mape: 0.11675548281949019
Testing Mape: 0.14449595466686968
chunk_size: 20000000000
Training R²: 0.8388250234555105
Testing R²: 0.7